In [2]:
import numpy as np

# 1. Max Palindrome 찾기  

In [ ]:
"""
# TestCase
>>> s = "kabccbadzdefgfeda"
>>> max_palindromes(s)
>>> ["k", "abccda", "dzd", "defgfed"]

>>> s = "kabccba dzbccbaza"
>>> max_palindromes(s)
>>> ["k", " ", "d", "zabccbaz", "aza"]
"""

In [71]:
"""
1. 문자열 s의 모든 substring을 찾습니다.
2. 각 substring이 palindrome인지 확인합니다.
3. palindrome이라면, 이 palindrome이 다른 palindrome의 substring인지 확인합니다.
4. substring이 아닌 palindrome 중에서 가장 긴 palindrome을 선택합니다.
"""

def palindrome(s: str):
    if s == s[::-1]:
        return True
    else:
        return False

# t가 s의 substring인지 check  
def substring(s, t):
    n = len(s)
    m = len(t)
    
    # 만약 길이 자체가 t가 더 크면 -> t는 s의 substring 아님
    if m > n:
        return False
    # s와 t 길이 차이만큼 돌건데 s를 t 길이만큼 돌면서 체크할 것
    for i in range(n-m+1):
        if s[i:i+m] == t:
            return True

    return False
    
# 주어진 문자열 s 내에서 가장 긴 palindrome들을 찾아 리스트로 반환하는 함수
def max_palindromes(s):
    max_pals = []
    
    # 모든 가능한 substring 뽑기
    for i in range(len(s)):
        for j in range(i, len(s)):
            sub = s[i:j+1]
            
            if palindrome(sub): # 현재 substring이 palindrome 일때 
                is_maximal = True # 현재 palindrome이 max라고 추정하고 시작
                
                for pal in max_pals:
                    # 만약 현재 palindrome이 기존 palindrome의 substring이라면
                    if substring(pal, sub) and pal != sub:
                        is_maximal = False
                        break
                
                # 만약 현재 palindrome이 기존 palindrome의 substring이 아니라면 후보에 넣기
                if is_maximal:
                    max_pals.append(sub)
    
    # palindrome 후보들 중 substring인 후보는 빼기 (filltering 작업)
    """
    max_pals = [pal for pal in max_pals if not any(substring(other, pal) for other in max_pals if other != pal)]
    """
    filtered_max_pals = []

    for i in range(len(max_pals)):
        is_substring = False
        for j in range(len(max_pals)):
            if i != j: # 자기 자신 제외
                # max_pals[i] 가 다른 palindrome의 substring인지 체크
                if substring(max_pals[j], max_pals[i]):
                    is_substring = True
                    break
                
        if is_substring == False:
            filtered_max_pals.append(max_pals[i])
    
    return filtered_max_pals

In [72]:
s = "kabccbadzdefgfeda"
max_palindromes(s)

['k', 'a', 'abccba', 'd', 'dzd', 'defgfed']


['k', 'abccba', 'dzd', 'defgfed']

In [73]:
s = "kabccba dzabccbaza"
max_palindromes(s)

['k', 'a', 'abccba', ' ', 'd', 'z', 'zabccbaz', 'aza']


['k', ' ', 'd', 'zabccbaz', 'aza']

# 2. Graph level partition 구하기 

In [ ]:
"""
connected undirected graph G
level_partition(G, s) 구현하기
    that partitions the nodes in the breadth-first tree of G rooted at s,
    list로 각 층마다 nodes들 반환하기 (층 inc order)
    
# TestCase
r, s, t, u, v = GNode('r'), GNode('s'), GNode('t'), GNode('u'), GNode('v')
w, x, y = GNode('w'), GNode('x'), GNode('y')
G = dict()
G[r], G[s], G[t], G[u], G[v] = [s, v], [w, r], [w, x, u], [t, x, y], [r]
G[w], G[x], G[y] = [s, t, x], [w, t, u, y], [x, u]
"""

In [116]:
"""
bfs(G, s) that performs a BFS algorithm (node s를 root로하여 Undirected graph G 만드는 함수)
level_partition(G, s) that uses bfs(G, s) (이전 함수 이용해서 level마다 node 담은 list 반환)
"""
import collections
# Node definition
class GNode:
    def __init__(self, id, color="W", d=0, p=None):
        self.id = id
        self.color = color
        self.distance = d
        self.parent = p
    
    def __str__(self): # object를 넣었을 때 어떤 걸 프린트할지
        return self.id

# (a) bfs(G,s) that performs a BFS algorithm on a connected undirectd graph G from the source node s
def bfs(G,s):
    # 그래프의 모든 노드 초기화
    for key in G:
        key.color = 'W'
        key.distance = -1
        key.parent = None
    
    # 시작 노드의 색상과 거리 설정
    s.color = "G"
    s.distance = 0
    Q = collections.deque() # BFS를 위한 queue 초기화
    Q.append(s) # source node를 먼저 queue에 넣고 시작
    
    while(len(Q)>0): #queue가 비어있지 않을 동안 계속
        cur = Q.popleft() # queue에서 cur node 꺼내기
        for adj in G[cur]: # 현재 노드의 인접 노드들에대해 돌기
            if adj.color == 'W': # 인접 노드가 방문 되지 않은 거라면 
                adj.color = 'G' # 방문하기 
                adj.distance = cur.distance + 1 # 거리 update
                adj.parent = cur # 인접 노드의 부모는 현재 노드 = 연결
                Q.append(adj) # 인접 노드들을 다시 queue에 넣기
                
        cur.color = 'B' #현재 노드를 검은색으로 바꾸어 완전히 탐색되었음을 표시

# # level_partition(G,s) 각 level마다 []로     
def level_partition(G,s):
    bfs(G, s) # bfs 실행
    distances = []
    for key in G: # 인접 리스트 돌면서 distance 저장하기
        if key.distance not in distances:
            distances.append(key.distance)
    distances.sort() # level order대로 

    ans = []
    while distances:
        partition = []
        cur = distances.pop(0)
        for key in G: # 해당 distance와 같은 distance 가지고 있는 노드들 []에 넣기
            if key.distance == cur:
                partition.append(key.id)
        
        ans.append(partition)
    return ans
        
        


In [117]:
r, s, t, u, v = GNode('r'), GNode('s'), GNode('t'), GNode('u'), GNode('v')
w, x, y = GNode('w'), GNode('x'), GNode('y')
G = dict()
G[r], G[s], G[t], G[u], G[v] = [s, v], [w, r], [w, x, u], [t, x, y], [r]
G[w], G[x], G[y] = [s, t, x], [w, t, u, y], [x, u]

In [118]:
level_partition(G,s)

[['s'], ['r', 'w'], ['t', 'v', 'x'], ['u', 'y']]